In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('fifa_players.csv')
df.head()

,name,full_name,birth_date,age,height_cm,weight_kgs,positions,nationality,overall_rating,potential,...,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle
0,L. Messi,Lionel Andrés Messi Cuccittini,6/24/1987,31,170.18,72.1,"CF,RW,ST",Argentina,94,94,...,94,48,22,94,94,75,96,33,28,26
1,C. Eriksen,Christian Dannemann Eriksen,2/14/1992,27,154.94,76.2,"CAM,RM,CM",Denmark,88,89,...,89,46,56,84,91,67,88,59,57,22
2,P. Pogba,Paul Pogba,3/15/1993,25,190.50,83.9,"CM,CAM",France,88,91,...,82,78,64,82,88,82,87,63,67,67
3,L. Insigne,Lorenzo Insigne,6/4/1991,27,162.56,59.0,"LW,ST",Italy,88,88,...,84,34,26,83,87,61,83,51,24,22
4,K. Koulibaly,Kalidou Koulibaly,6/20/1991,27,187.96,88.9,CB,Senegal,88,91,...,15,87,88,24,49,33,80,91,88,87


In [ ]:
'''
Data shape
'''
df.shape

(17954, 51)

In [ ]:
'''
Check clean or not
'''

df.isna().sum()

name                                 0
full_name                            0
birth_date                           0
age                                  0
height_cm                            0
weight_kgs                           0
positions                            0
nationality                          0
overall_rating                       0
potential                            0
value_euro                         255
wage_euro                          246
preferred_foot                       0
international_reputation(1-5)        0
weak_foot(1-5)                       0
skill_moves(1-5)                     0
body_type                            0
release_clause_euro               1837
national_team                    17097
national_rating                  17097
national_team_position           17097
national_jersey_number           17097
crossing                             0
finishing                            0
heading_accuracy                     0
short_passing            

In [ ]:
df = df.drop(['national_team', 'national_rating', 'national_team_position', 'national_jersey_number'], axis=1)
df = df.dropna()
df.shape

(16117, 47)

In [ ]:
'''
Encode Data
'''

le = LabelEncoder()

for col in df.select_dtypes(include=['object']).columns:
    df[col] = le.fit_transform(df[col])

In [ ]:
'''
Memisalkan variabel dependen dan independen
'''

X = df.drop(["overall_rating"], axis = 1) # Variabel independen
y = df["overall_rating"]  # Variabel dependen / Label / Target

In [ ]:
'''
Memisahkan data training dan data validation
'''

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
'''
Mencari model terbaik
'''

regression = [
    ('LinReg', LinearRegression()),
    ('PolyReg', PolynomialFeatures()),
    ('Ridge', Ridge(alpha=1.0)),
    ('Lasso', Lasso(alpha=1.0)),
    ('SVR', SVR()),
    ('DT', DecisionTreeRegressor()),
    ('RF', RandomForestRegressor())
]

result_reg = pd.DataFrame(columns=['Model', 'MSE', 'R2'])

for name, model in regression:
    if name == 'PolyReg':
        poly = PolynomialFeatures(degree=2)
        X_poly = poly.fit_transform(X_train)
        model = LinearRegression()
        mse = cross_val_score(model, X_poly, y_train, cv=5, scoring='neg_mean_squared_error').mean()
        r2 = cross_val_score(model, X_poly, y_train, cv=5, scoring='r2').mean()
    else:
        model.fit(X_train, y_train)
        mse = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()
        r2 = cross_val_score(model, X_train, y_train, cv=5, scoring='r2').mean()

    result_reg.loc[len(result_reg)] = [name, mse, r2]

result_reg.sort_values(by='R2', ascending=False)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.617e+04, tolerance: 6.459e+01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.139e+04, tolerance: 5.154e+01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.467e+04, tolerance: 5.180e

,Model,MSE,R2
6,RF,-0.198700,0.995950
5,DT,-0.480963,0.990638
1,PolyReg,-2.052905,0.958966
2,Ridge,-4.070195,0.918694
0,LinReg,-4.070199,0.918694
3,Lasso,-4.527543,0.909545
4,SVR,-10.482681,0.790588


In [ ]:
df.shape

(16117, 47)

In [ ]:
'''
Feature Selection menggunakan RFE
'''

model = RandomForestRegressor()
rfe = RFE(model, n_features_to_select=30)
rfe.fit(X_train, y_train)

RFE(estimator=RandomForestRegressor(), n_features_to_select=30)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
y_pred = rfe.predict(X_val)

r2 = r2_score(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)

print("R2 Score:", r2)
print("Mean Squared Error:", mse)

R2 Score: 0.9966659450631368
Mean Squared Error: 0.1667100496277916


In [ ]:
selected_columns = X.columns[rfe.support_]

print("Selected Columns:")
print(selected_columns)

Selected Columns:
Index(['full_name', 'birth_date', 'age', 'potential', 'value_euro',
       'wage_euro', 'release_clause_euro', 'crossing', 'finishing',
       'heading_accuracy', 'volleys', 'dribbling', 'curve',
       'freekick_accuracy', 'ball_control', 'acceleration', 'agility',
       'reactions', 'shot_power', 'stamina', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'composure',
       'marking', 'standing_tackle', 'sliding_tackle'],
      dtype='object')


In [ ]:
selected_features = rfe.support_

In [ ]:
X_train_features = X_train[X.columns[rfe.support_]]
X_test_features = X_val[X.columns[rfe.support_]]

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model = RandomForestRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_features, y_train)

print("Parameter terbaik:", grid_search.best_params_)
print("Akurasi validasi terbaik:", grid_search.best_score_)